In [1]:
def get_parameters(filename):
    with open(filename, "r") as f:
        vehicle_size = [int(size) for size in f.readline().replace("\n", "").split(" ")]
        nb_items = int(f.readline().replace("\n", ""))
        items_size = []
        for line in f.readlines():
            items_size.append([int(size) for size in line.replace("\n", "").split(" ")])
    
    assert nb_items == len(items_size)
    return vehicle_size, nb_items, items_size

truck_dimensions, _, item_dimensions = get_parameters("input")
truck_dimensions, item_dimensions

([250, 150, 170],
 [[70, 150, 140],
  [20, 90, 50],
  [50, 20, 20],
  [190, 170, 190],
  [40, 150, 110],
  [100, 130, 60],
  [110, 190, 160],
  [30, 20, 150],
  [80, 110, 120],
  [30, 160, 50],
  [90, 10, 130],
  [30, 150, 130],
  [80, 140, 30],
  [100, 160, 40],
  [150, 100, 70],
  [50, 160, 90],
  [180, 40, 10],
  [160, 120, 130],
  [170, 140, 190],
  [160, 20, 10],
  [30, 80, 190],
  [120, 30, 100],
  [30, 160, 130],
  [80, 160, 190],
  [100, 30, 70],
  [120, 180, 40],
  [30, 90, 50],
  [100, 110, 90],
  [40, 40, 100],
  [130, 110, 100],
  [180, 50, 120],
  [140, 60, 10],
  [40, 90, 140],
  [50, 90, 130],
  [80, 120, 60],
  [130, 20, 70],
  [10, 100, 80],
  [20, 190, 170],
  [60, 150, 120],
  [110, 40, 160],
  [90, 170, 190],
  [110, 60, 160],
  [130, 160, 130],
  [160, 20, 190],
  [170, 10, 140],
  [10, 40, 10],
  [90, 20, 10],
  [80, 80, 20],
  [40, 140, 130],
  [60, 30, 50],
  [200, 170, 130],
  [180, 60, 20],
  [60, 50, 130],
  [30, 200, 70],
  [180, 100, 120],
  [140, 150, 140]

In [2]:
#truck_dimensions = [120, 330, 230]
#item_dimensions = [[80, 30, 230], [50, 160, 50], [200, 200, 10], [10, 10, 10], [150, 10, 10], [500, 500, 500]]
#nb_items = len(item_dimensions)

#item_dimensions = item_dimensions[:12]

In [3]:
import pulp

def solve_packing_problem(truck_dims, items, weight_num_items, weight_volume):
    model: pulp.LpProblem = pulp.LpProblem("Truck_packing_problem", pulp.LpMaximize)
    num_items = len(items)

    x = pulp.LpVariable.dicts("x", range(num_items), cat='Binary')
    pos = pulp.LpVariable.dicts("pos", [(i, j) for i in range(num_items) for j in range(3)], 
                                lowBound=0, cat='Continuous')
    orientation = pulp.LpVariable.dicts("orientation", 
                                        [(i, j) for i in range(num_items) for j in range(6)], 
                                        cat='Binary')

    total_volume = sum(items[i][0] * items[i][1] * items[i][2] * x[i] for i in range(num_items))
    total_items = sum(x[i] for i in range(num_items))
    model += weight_num_items * total_items + weight_volume * total_volume

    for i in range(num_items):
        model += sum(orientation[(i, j)] for j in range(6)) == x[i]
        for j in range(3):
            real_dim = sum(orientation[(i, k)] * items[i][k % 3] for k in range(6))
            model += pos[(i, j)] + real_dim <= truck_dims[j]

    for i in range(num_items):
        real_dim_i_x = sum(orientation[(i, k)] * items[i][k % 3] for k in range(6))
        real_dim_i_y = sum(orientation[(i, k)] * items[i][(k + 1) % 3] for k in range(6))
        real_dim_i_z = sum(orientation[(i, k)] * items[i][(k + 2) % 3] for k in range(6))

        model += pos[(i, 0)] + real_dim_i_x <= truck_dims[0]
        model += pos[(i, 1)] + real_dim_i_y <= truck_dims[1]
        model += pos[(i, 2)] + real_dim_i_z <= truck_dims[2]

        for j in range(i + 1, num_items):
            real_dim_j_x = sum(orientation[(j, k)] * items[j][k % 3] for k in range(6))
            real_dim_j_y = sum(orientation[(j, k)] * items[j][(k + 1) % 3] for k in range(6))
            real_dim_j_z = sum(orientation[(j, k)] * items[j][(k + 2) % 3] for k in range(6))

            left = pulp.LpVariable(f"left_{i}_{j}", cat='Binary')
            right = pulp.LpVariable(f"right_{i}_{j}", cat='Binary')
            above = pulp.LpVariable(f"above_{i}_{j}", cat='Binary')
            below = pulp.LpVariable(f"below_{i}_{j}", cat='Binary')
            front = pulp.LpVariable(f"front_{i}_{j}", cat='Binary')
            back = pulp.LpVariable(f"back_{i}_{j}", cat='Binary')

            model += pos[(i, 0)] + real_dim_i_x <= pos[(j, 0)] + truck_dims[0]*(1 - left)
            model += pos[(j, 0)] + real_dim_j_x <= pos[(i, 0)] + truck_dims[0]*(1 - right)
            model += pos[(i, 1)] + real_dim_i_y <= pos[(j, 1)] + truck_dims[1]*(1 - below)
            model += pos[(j, 1)] + real_dim_j_y <= pos[(i, 1)] + truck_dims[1]*(1 - above)
            model += pos[(i, 2)] + real_dim_i_z <= pos[(j, 2)] + truck_dims[2]*(1 - back)
            model += pos[(j, 2)] + real_dim_j_z <= pos[(i, 2)] + truck_dims[2]*(1 - front)

            model += left + right + above + below + front + back >= x[i] + x[j] - 1


    model.solve()
    packed_items = [i for i in range(num_items) if pulp.value(x[i]) == 1]
    positions = {i: (pulp.value(pos[(i, 0)]), pulp.value(pos[(i, 1)]), pulp.value(pos[(i, 2)])) 
                 for i in packed_items}
    orientations = {i: [pulp.value(orientation[(i, j)]) for j in range(6)] 
                    for i in packed_items}
    return packed_items, positions, orientations

weight_num_items = 1
weight_volume = 5

packed_items, positions, orientations = solve_packing_problem(truck_dimensions, item_dimensions, weight_num_items, weight_volume)
packed_items, positions, orientations

([0, 1, 2, 4, 5, 7, 8, 9, 10, 11],
 {0: (150.0, 0.0, 30.0),
  1: (40.0, 0.0, 0.0),
  2: (100.0, 30.0, 10.0),
  4: (0.0, 0.0, 60.0),
  5: (0.0, 20.0, 0.0),
  7: (230.0, 0.0, 140.0),
  8: (40.0, 30.0, 60.0),
  9: (100.0, 0.0, 10.0),
  10: (120.0, 20.0, 0.0),
  11: (220.0, 0.0, 10.0)},
 {0: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0],
  1: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  2: [0.0, 1.0, 0.0, 0.0, 0.0, 0.0],
  4: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  5: [0.0, 0.0, 0.0, 1.0, 0.0, 0.0],
  7: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  8: [0.0, 0.0, 0.0, 0.0, 1.0, 0.0],
  9: [0.0, 0.0, 0.0, 0.0, 0.0, 1.0],
  10: [0.0, 0.0, 1.0, 0.0, 0.0, 0.0],
  11: [1.0, 0.0, 0.0, 0.0, 0.0, 0.0]})

In [4]:
def convert_to_output_format(packed_items, positions, orientations, items, file_path):
    with open(file_path, 'w') as file:
        file.write("SAT\n")
        for i in packed_items:
            x1, y1, z1 = positions[i]
            dim_x = sum(orientations[i][k] * items[i][k % 3] for k in range(6))
            dim_y = sum(orientations[i][k] * items[i][(k + 1) % 3] for k in range(6))
            dim_z = sum(orientations[i][k] * items[i][(k + 2) % 3] for k in range(6))
            x2, y2, z2 = x1 + dim_x, y1 + dim_y, z1 + dim_z

            file.write(f"0 {int(x1)} {int(y1)} {int(z1)} {int(x2)} {int(y2)} {int(z2)}\n")

file_path = 'output.txt'

convert_to_output_format(packed_items, positions, orientations, item_dimensions, file_path)

In [5]:
f"python visualize.py output.txt --truck-dimensions \"{'x'.join([str(dim) for dim in truck_dimensions])}\""

'python visualize.py output.txt --truck-dimensions "250x150x170"'

In [6]:
from ortools.sat.python import cp_model

def solve_packing_problem_cp(truck_dims, items):
    model = cp_model.CpModel()

    num_items = len(items)
    x = [model.NewIntVar(0, truck_dims[0], f'x[{i}]') for i in range(num_items)]
    y = [model.NewIntVar(0, truck_dims[1], f'y[{i}]') for i in range(num_items)]
    z = [model.NewIntVar(0, truck_dims[2], f'z[{i}]') for i in range(num_items)]
    used = [model.NewBoolVar(f'used[{i}]') for i in range(num_items)]

    for i in range(num_items):
        model.Add(x[i] + items[i][0] <= truck_dims[0]).OnlyEnforceIf(used[i])
        model.Add(y[i] + items[i][1] <= truck_dims[1]).OnlyEnforceIf(used[i])
        model.Add(z[i] + items[i][2] <= truck_dims[2]).OnlyEnforceIf(used[i])

    for i in range(num_items):
        for j in range(i + 1, num_items):
            left = model.NewBoolVar(f'left_{i}_{j}')
            right = model.NewBoolVar(f'right_{i}_{j}')
            above = model.NewBoolVar(f'above_{i}_{j}')
            below = model.NewBoolVar(f'below_{i}_{j}')
            front = model.NewBoolVar(f'front_{i}_{j}')
            back = model.NewBoolVar(f'back_{i}_{j}')

            model.Add(x[i] + items[i][0] <= x[j]).OnlyEnforceIf(left)
            model.Add(x[j] + items[j][0] <= x[i]).OnlyEnforceIf(right)
            model.Add(y[i] + items[i][1] <= y[j]).OnlyEnforceIf(below)
            model.Add(y[j] + items[j][1] <= y[i]).OnlyEnforceIf(above)
            model.Add(z[i] + items[i][2] <= z[j]).OnlyEnforceIf(back)
            model.Add(z[j] + items[j][2] <= z[i]).OnlyEnforceIf(front)

            model.AddBoolOr([left, right, above, below, front, back, used[i].Not(), used[j].Not()])

    model.Maximize(sum(used))

    solver = cp_model.CpSolver()
    status = solver.Solve(model)

    if status == cp_model.OPTIMAL:
        packed_items = [i for i in range(num_items) if solver.Value(used[i]) == 1]
        positions = [(solver.Value(x[i]), solver.Value(y[i]), solver.Value(z[i])) for i in packed_items]
        return packed_items, positions
    else:
        return [], []

items, positions = solve_packing_problem_cp(truck_dimensions, item_dimensions)
items, positions

([0, 1, 2, 4, 7, 8, 10, 11],
 [(0, 0, 0),
  (70, 0, 0),
  (90, 10, 0),
  (180, 0, 0),
  (90, 120, 0),
  (70, 10, 50),
  (90, 0, 0),
  (220, 0, 0)])

In [8]:
from random import randint

with open("output_cp.txt", 'w') as file:
    file.write("SAT\n")
    for i, item_id in enumerate(items):
        file.write(f"{0} {positions[i][0]} {positions[i][1]} {positions[i][2]} {positions[i][0] + item_dimensions[item_id][0]} {positions[i][1] + item_dimensions[item_id][1]} {positions[i][2] + item_dimensions[item_id][2]}\n")
f"python visualize.py output_cp.txt --truck-dimensions \"{'x'.join([str(dim) for dim in truck_dimensions])}\""

'python visualize.py output_cp.txt --truck-dimensions "250x150x170"'